Presentado por: Diego Quisi

# Entornos de soporte al desarrollo de simulaciones: Simpy - Parte 2


## Generacion de graficar de atencion. 

En base a la simulacion anterior generar una grafica de los autos atendidos.

## Práctica  Sympy 2

En base a la simulacion, generar una simulacion lo mas apegado a la realidad del Covid-19, para ello obtener informacion del numero de consultorios y camas disponibles en un hospital del Ecuador. Parametrizar la simulacion para ingresar el numero de pacientes, el numero de consultorios y el numero de camas y algun otro parametro que considere oportuno.

Finalmente generar las siguientes metricas :
- Tiempo de espera promedio para cada sala.
- Tiempo promedio total que requiere un paciente para ser atendido desde cero.
- Cuántos pacientes se atendieron.
- Cuántos pacientes se quedaron sin atender y fallecieron.

Hospital seleccionado: Luis G. Dávila (Carchi)
  - Camas:          492 (https://www.salud.gob.ec/hospital-general-guasmo-sur/)
  - Consultorios:   145 (https://www.salud.gob.ec/hospital-general-guasmo-sur/)
  - Tiempo de espera promedio para cada sala: en promedio es de 20 minutos segun un estudio realizado y disponible en (https://dspace.ups.edu.ec/bitstream/123456789/5059/1/UPS-CT002680.pdf)
  - Tiempo de consulta: en promedio es de 15 minutos de acuerdo al siguiente articulo: https://dspace.ups.edu.ec/bitstream/123456789/5059/1/UPS-CT002680.pdf

In [33]:
import simpy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


# En una nota de prensa realizada (https://www.edicionmedica.ec/secciones/gestion/hospital-guasmo-sur-supera-las-100-000-atenciones-desde-su-apertura-91185)
# habla de 100000 pacientes atendidos en un tiempo de 9 meses
# asi que basandose en esta informacion
#obtendremos el numero de pacientes diarios
# 100000/(9*30) = 370 Es un aproximado a lo que se atenderia por dia en dicho hospital
# ademas el hospital atiende las 24 horas del dia lo cual significan 1440 minutos
# entonces para calcular el intervalo de llegada aproximado divido los 1440
# para 370(pacientes por dia) lo cual da 3.89min aproximadamente
# lo que redondeando representa un 4
intervalo_llegada = 4

#La duracion sera 1440 debido a que mantiene su atencion las 24h
duracion = 1440

tiempo_consulta = 15
tiempo_espera = 20
consultorios = 145
camas = 492

pacientes_total = 0
hospitalizados = 0
atendidos = 0
diagnosticados = 0

tiempos_espera = {}
tiempos_llegada = {}
tiempos_atencion = {} 
tiempos_atencion_cero = {}

class Hospital(object):
    def __init__(self, environment, consultorios, camas, tiempo_consulta, tiempo_espera):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Recurso consultorios
        self.consultorios = simpy.Resource(environment, consultorios)
        # Recurso camas
        self.camas = simpy.Resource(environment, camas)
        # Tiempo de consulta
        self.tiempo_consulta = tiempo_consulta
        self.tiempo_espera = tiempo_espera
        
    # Método que dicta el tiempo que el paciente espera en la sala    
    def espera_paciente(self, paciente):
        espera = random.randint(tiempo_espera-intervalo_llegada, tiempo_espera+intervalo_llegada)
        yield self.env.timeout(espera) 
        
    # Método que dicta el tiempo de l aduración de la consulta del paciente
    def atender_paciente(self, paciente):
        yield self.env.timeout(random.randint(tiempo_consulta-intervalo_llegada, tiempo_consulta+intervalo_llegada))
        
def llega_paciente(env, paciente, hospital):
    # Usamos el reloj de la simulacion (env.now()) para indicar a la
    # hora que llega el vehiculo con el nombre pasado como parametro
    
    global pacientes_total
    global tiempos_llegada
    
    pacientes_total += 1
    tiempos_llegada[paciente] = env.now
    print("Llega el paciente "+str(paciente)+" a las "+str(number_to_time((env.now/60))))

    # Especificamos que vamos a usar un recurso (Resource) que representa un consultorio
    with hospital.consultorios.request() as consultorio:
        yield env.process(hospital.espera_paciente(paciente))
        # Ocupamos el consultorio
        yield consultorio
        global atendidos
        
        #usamos try para para agarrar una excepcion
        try:
            tiempos_espera[paciente] = env.now - tiempos_llegada[paciente]
        except:
            a=1 
        
        print("El paciente ",paciente," se atiende a las ", number_to_time((env.now/60)))
        atendidos+=1 
        global diagnosticados
         
        yield env.process(hospital.atender_paciente(paciente)) 
        print("El paciente ",paciente," sale de consulta a las ", number_to_time((env.now/60)))
        
        diagnosticados += 1
        
    
    with hospital.camas.request() as cama:
        yield cama
        global hospitalizados
        global tiempos_atencion
        hospitalizados +=1 
        print("el paciente ", paciente," se hospitaliza a las ",str(number_to_time(env.now/60)))  
        try:
            tiempos_atencion_cero[paciente] = env.now - tiempos_llegada[paciente]
        except:
            a=1
        yield env.timeout(1440)
        
def number_to_time(number):
    x, y = math.modf(round(number,2)) 
    return round(round(y + (x * 60 /100),2),2)
        
        
def ejecutar_simulacion(env, consultorios, camas, tiempo_consulta, tiempo_espera, intervalo):
    hospital = Hospital(env, consultorios, camas, tiempo_consulta, tiempo_espera)
    # Creamos 5 llegadas de vehiculos iniciales
    for i in range(17):
        env.process(llega_paciente(env, i, hospital))
    # Ejecutamos la simulacion
    while True:
        yield env.timeout(random.randint(intervalo-intervalo_llegada, intervalo+intervalo_llegada))
        i+=1
        # Mientras se lavan los vehiculos generamos mas vehiculos
        env.process(llega_paciente(env,i,hospital))
        

env=simpy.Environment()
env.process(ejecutar_simulacion(env, consultorios, camas, tiempo_consulta, tiempo_espera, intervalo_llegada))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = duracion)


 


Llega el paciente 0 a las 0.0
Llega el paciente 1 a las 0.0
Llega el paciente 2 a las 0.0
Llega el paciente 3 a las 0.0
Llega el paciente 4 a las 0.0
Llega el paciente 5 a las 0.0
Llega el paciente 6 a las 0.0
Llega el paciente 7 a las 0.0
Llega el paciente 8 a las 0.0
Llega el paciente 9 a las 0.0
Llega el paciente 10 a las 0.0
Llega el paciente 11 a las 0.0
Llega el paciente 12 a las 0.0
Llega el paciente 13 a las 0.0
Llega el paciente 14 a las 0.0
Llega el paciente 15 a las 0.0
Llega el paciente 16 a las 0.0
Llega el paciente 17 a las 0.07
Llega el paciente 18 a las 0.08
Llega el paciente 19 a las 0.15
El paciente  11  se atiende a las  0.16
El paciente  16  se atiende a las  0.16
El paciente  2  se atiende a las  0.18
El paciente  10  se atiende a las  0.18
El paciente  1  se atiende a las  0.19
El paciente  5  se atiende a las  0.19
El paciente  14  se atiende a las  0.19
El paciente  15  se atiende a las  0.19
El paciente  6  se atiende a las  0.2
El paciente  12  se atiende a la

el paciente  85  se hospitaliza a las  5.0
El paciente  95  se atiende a las  5.01
El paciente  90  se atiende a las  5.02
Llega el paciente 99 a las 5.03
El paciente  92  se atiende a las  5.05
El paciente  96  se atiende a las  5.05
El paciente  93  se atiende a las  5.06
El paciente  91  se atiende a las  5.07
El paciente  94  se atiende a las  5.08
El paciente  89  sale de consulta a las  5.08
el paciente  89  se hospitaliza a las  5.08
El paciente  87  sale de consulta a las  5.09
el paciente  87  se hospitaliza a las  5.09
Llega el paciente 100 a las 5.11
El paciente  88  sale de consulta a las  5.15
El paciente  95  sale de consulta a las  5.15
el paciente  88  se hospitaliza a las  5.15
el paciente  95  se hospitaliza a las  5.15
Llega el paciente 101 a las 5.16
El paciente  98  se atiende a las  5.16
El paciente  97  se atiende a las  5.17
El paciente  93  sale de consulta a las  5.17
el paciente  93  se hospitaliza a las  5.17
Llega el paciente 102 a las 5.18
El paciente  90 

El paciente  165  se atiende a las  9.44
Llega el paciente 172 a las 9.46
El paciente  164  sale de consulta a las  9.48
el paciente  164  se hospitaliza a las  9.48
El paciente  168  se atiende a las  9.5
Llega el paciente 173 a las 9.52
El paciente  167  se atiende a las  9.52
El paciente  170  se atiende a las  9.54
El paciente  169  se atiende a las  9.58
El paciente  171  se atiende a las  9.58
Llega el paciente 174 a las 9.59
El paciente  166  sale de consulta a las  9.59
el paciente  166  se hospitaliza a las  9.59
Llega el paciente 175 a las 10.01
El paciente  168  sale de consulta a las  10.01
el paciente  168  se hospitaliza a las  10.01
El paciente  165  sale de consulta a las  10.03
El paciente  167  sale de consulta a las  10.03
el paciente  165  se hospitaliza a las  10.03
el paciente  167  se hospitaliza a las  10.03
Llega el paciente 176 a las 10.04
Llega el paciente 177 a las 10.04
El paciente  170  sale de consulta a las  10.05
el paciente  170  se hospitaliza a las  

el paciente  240  se hospitaliza a las  14.25
Llega el paciente 248 a las 14.27
Llega el paciente 249 a las 14.28
El paciente  244  se atiende a las  14.28
El paciente  241  sale de consulta a las  14.29
el paciente  241  se hospitaliza a las  14.29
El paciente  245  se atiende a las  14.31
Llega el paciente 250 a las 14.32
El paciente  242  sale de consulta a las  14.32
el paciente  242  se hospitaliza a las  14.32
Llega el paciente 251 a las 14.35
Llega el paciente 252 a las 14.37
El paciente  243  sale de consulta a las  14.39
el paciente  243  se hospitaliza a las  14.39
El paciente  246  se atiende a las  14.4
Llega el paciente 253 a las 14.43
El paciente  245  sale de consulta a las  14.44
el paciente  245  se hospitaliza a las  14.44
El paciente  247  se atiende a las  14.45
El paciente  248  se atiende a las  14.45
El paciente  244  sale de consulta a las  14.47
el paciente  244  se hospitaliza a las  14.47
Llega el paciente 254 a las 14.49
Llega el paciente 255 a las 14.5
El p

Llega el paciente 314 a las 18.24
El paciente  305  se atiende a las  18.25
Llega el paciente 315 a las 18.28
El paciente  304  sale de consulta a las  18.28
el paciente  304  se hospitaliza a las  18.28
El paciente  309  se atiende a las  18.3
El paciente  306  sale de consulta a las  18.31
el paciente  306  se hospitaliza a las  18.31
El paciente  308  se atiende a las  18.32
El paciente  310  se atiende a las  18.33
Llega el paciente 316 a las 18.34
El paciente  303  sale de consulta a las  18.35
El paciente  307  sale de consulta a las  18.35
el paciente  303  se hospitaliza a las  18.35
el paciente  307  se hospitaliza a las  18.35
Llega el paciente 317 a las 18.36
El paciente  305  sale de consulta a las  18.37
el paciente  305  se hospitaliza a las  18.37
El paciente  312  se atiende a las  18.4
El paciente  311  se atiende a las  18.42
Llega el paciente 318 a las 18.44
El paciente  310  sale de consulta a las  18.44
el paciente  310  se hospitaliza a las  18.44
El paciente  315

El paciente  359  se atiende a las  21.37
El paciente  356  sale de consulta a las  21.38
el paciente  356  se hospitaliza a las  21.38
Llega el paciente 364 a las 21.39
El paciente  360  se atiende a las  21.41
El paciente  357  sale de consulta a las  21.42
el paciente  357  se hospitaliza a las  21.42
El paciente  361  se atiende a las  21.46
El paciente  358  sale de consulta a las  21.46
el paciente  358  se hospitaliza a las  21.46
Llega el paciente 365 a las 21.47
Llega el paciente 366 a las 21.47
Llega el paciente 367 a las 21.49
El paciente  360  sale de consulta a las  21.52
el paciente  360  se hospitaliza a las  21.52
El paciente  362  se atiende a las  21.53
El paciente  359  sale de consulta a las  21.54
el paciente  359  se hospitaliza a las  21.54
Llega el paciente 368 a las 21.55
El paciente  363  se atiende a las  21.55
Llega el paciente 369 a las 22.01
El paciente  364  se atiende a las  22.01
El paciente  361  sale de consulta a las  22.02
el paciente  361  se hospi

In [34]:

mean_tespera = int(np.round(np.mean(np.array([tiempos_espera[x] for x in tiempos_espera]))))#[0 for i in range(10)]#np.array(tiempos_espera.items())
print("tiempo promedio de espera es: ",mean_tespera)

mean_tespera_zero = int(np.round(np.mean(np.array([tiempos_atencion_cero[x] for x in tiempos_atencion_cero]))))#[0 for i in range(10)]#np.array(tiempos_espera.items())
print("tiempo promedio de espera desde cero es: ",mean_tespera)

fallecidos = pacientes_total - (abs(hospitalizados - atendidos)+hospitalizados)
print("El total de fallecidos es: ",fallecidos)

total_atendidos = abs(atendidos-hospitalizados)+hospitalizados
print("El numero de pacientes atendidos es: ",total_atendidos)


tiempo promedio de espera es:  20
tiempo promedio de espera desde cero es:  20
El total de fallecidos es:  4
El numero de pacientes atendidos es:  390



## Referencias
[1] Matloff, N. (2008). Introduction to Discrete-Event Simulation and the SimPy Language.

[2] Team Simpy (2017). SimPy Documentation, Release 3.0.10, 2017. URL: https://media.readthedocs.org/pdf/simpy/latest/simpy.pdf 